<h2 align='center'>PROJECT 4 - ACCRE</h2>

#### Using just the fullsample.csv jobs dataset:

### 1. Calculate some descriptive statistics for how many jobs per hour are being completed. What does the completions per hour look like over the time span of the dataset? Are there weekly trends, and has it been increasing over the last year?

In [1]:
from dateutil.parser import parse
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
full_sample = pd.read_csv('../data/fullsample.csv',
                          usecols = [0,1,2,3,
                                     6,7,
                                     10,11])

In [3]:
full_sample.info()
display(full_sample.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7395885 entries, 0 to 7395884
Data columns (total 8 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   JOBID      object
 1   STATE      object
 2   BEGIN      object
 3   END        object
 4   REQTIME    object
 5   USEDTIME   object
 6   PARTITION  object
 7   EXITCODE   object
dtypes: object(8)
memory usage: 451.4+ MB


,JOBID,STATE,BEGIN,END,REQTIME,USEDTIME,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,10:04:00,67-22:14:22,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,30-00:00:00,30-00:00:23,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,30-00:00:00,30-00:00:14,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,29-04:00:00,29-04:00:04,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,24-09:00:00,24-09:00:17,production,0:0


In [4]:
full_sample['BEGIN'] = pd.to_datetime(full_sample['BEGIN'],
                                      errors = 'coerce')

In [5]:
full_sample['END'] = pd.to_datetime(full_sample['END'],
                                    errors = 'coerce')

In [6]:
full_sample = full_sample.sort_values('END')

In [7]:
full_sample[full_sample['END'].isnull()]

,JOBID,STATE,BEGIN,END,REQTIME,USEDTIME,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31 22:15:00,NaT,10:04:00,67-22:14:22,production,0:0
7343,31874232_1949,RUNNING,2021-08-31 23:55:05,NaT,00:15:00,36-20:34:17,production,0:0
7650,31874232_2461,RUNNING,2021-08-31 23:58:07,NaT,00:15:00,36-20:31:15,production,0:0
7676,31874232_2487,RUNNING,2021-08-31 23:58:07,NaT,00:15:00,36-20:31:15,production,0:0
8621,31874232_3432,RUNNING,2021-09-01 00:03:07,NaT,00:15:00,36-20:26:15,production,0:0
...,...,...,...,...,...,...,...,...
927267,32922493,RUNNING,2021-10-07 09:13:25,NaT,2-00:00:00,11:15:57,nogpfs,0:0
927268,32922526,RUNNING,2021-10-07 09:18:25,NaT,2-00:00:00,11:10:57,nogpfs,0:0
927269,32922527,RUNNING,2021-10-07 09:25:25,NaT,2-00:00:00,11:03:57,nogpfs,0:0
927270,32922542,RUNNING,2021-10-07 09:36:25,NaT,2-00:00:00,10:52:57,nogpfs,0:0


In [8]:
# MOVING COUNT OF JOBS PER HOUR -- MUST USE "notnull()" AS IN HABEEB'S CODE
#   OR YOU WILL GET "'END' MUST BE MONOTONIC" ERROR 

# full_sample['jobs_per_hour'] =

full_sample[full_sample['END'].notnull()]\
                                          .rolling('1h', 
                                                   on = 'END')['JOBID']\
                                          .count()
# display(full_sample.head())

6640577    1.0
6640635    2.0
6640627    3.0
6640651    4.0
6640654    5.0
          ... 
927386     9.0
927387     9.0
927388     8.0
1491977    9.0
1491978    9.0
Name: JOBID, Length: 7395234, dtype: float64

In [ ]:
# THIS TAKES WAY TOO LONG TO GRAPH - TODO 10/30

# full_sample[full_sample['END'].notnull()].plot(kind = 'line',
#                                               x = 'END',
#                                               y = 'jobs_per_hour');

In [10]:
# full_sample[full_sample['END'].notnull()]['END'].dt.date

In [ ]:
# IS describe() USEFUL HERE? -- TODO 10/30

# full_sample\
#            .groupby('END'.dt.date)['jobs_per_hour']\
#            .describe()

In [ ]:
# GROUP BY YEAR-MONTH-DAY-HOUR?

# jobs_by_day_hour = (full_sample[full_sample['END'].notnull()]
#                               .groupby(['END'])
#                               .rolling('1h', 
#                                        on = 'END')['JOBID']
#                               .count()
#                               .reset_index()
#                               .set_index('END')
#                   )
# jobs_by_day_hour

In [ ]:
job_state_counts = full_sample\
                               .groupby('STATE')['JOBID']\
                               .count()\
                               .sort_values(ascending=False)
                    #          .describe()
job_state_counts

In [ ]:
# full_sample['STATE'].unique()

In [ ]:
# zero_exits = full_sample[(full_sample['STATE']!='COMPLETED') & (full_sample['EXITCODE'] == '0:0')]
# print(zero_exits['STATE'].unique())

#### Using just the fullsample.csv jobs dataset:

### 2. Does the job state affect completions per hour? i.e. if I only look at jobs with exit code 0:0 in the "COMPLETED" state, is that a similar number of completions per hour as with all jobs, failed or cancelled? This will indicate if the load on the scheduler is by user design or is a result of users not sufficiently testing their jobs before submitting very large arrays. We also expect that most job completions will be in the "production" partition, but is this actually true?

In [ ]:
'''
EXITCODE 0:0            = SUCCESS
         0:15, 0:9, 0:2 = CANCELLED
         1:0, 2.0       = FAILED
'''
exit_code_counts = full_sample\
                               .groupby('EXITCODE')['JOBID']\
                               .count()\
                               .sort_values(ascending=False)
exit_code_counts

In [ ]:
# MOVING COUNT GROUPING BY EXIT CODE -- 

group_by_exit_code = (full_sample    # full_sample[full_sample['END'].notnull()]
                                 .groupby('EXITCODE')
                                 .rolling('1h', 
                                          on = 'END')['JOBID']
                                 .count()
                                 .reset_index()
                                 .set_index('END')
                      )
group_by_exit_code

#### Using the two log files:

### 3.	Calculate some descriptive statistics about how often the scheduler was unresponsive, how long these periods of time were, and create a time series plot of when the scheduler was having difficulties.

In [ ]:
ce5_raw = pd.read_table("../data/slurm_wrapper_ce5.log", 
                        engine='python',
                        sep=" - ", 
                       # skiprows=[0], 
                       # header=0,
                        names=['ce5_DateTime', 
                               'ce5_User', 
                               'ce5_Retry',
                               'ce5_Time_Spent',
                               'ce5_Return_Code',
                               'ce5_Command']
                       ) 
                        # parse_dates=[1]).rename(columns=lambda x: x.strip())

In [ ]:
ce5_raw.info()
display(ce5_raw.head())

In [ ]:
ce5_raw['DateTime'] = pd.to_datetime(ce5_raw.DateTime)

In [ ]:
ce5_hrs_range = ce5_raw['DateTime'].max() - ce5_raw['DateTime'].min()
display(ce5_hrs_range)

In [ ]:
ce6_raw = pd.read_table("../data/slurm_wrapper_ce6.log", 
                        engine='python',
                        sep=" - ", 
                       # skiprows=[0], 
                       # header=0,
                        names=['ce6_DateTime', 
                               'ce6_User', 
                               'ce6_Retry',
                               'ce6_Time_Spent',
                               'ce6_Return_Code',
                               'ce6_Command']
                       ) 
                        # parse_dates=[1]).rename(columns=lambda x: x.strip())

In [ ]:
ce6_raw.info()
display(ce6_raw.head())

In [ ]:
ce6_raw['DateTime'] = pd.to_datetime(ce6_raw.DateTime)

In [ ]:
ce6_hrs_range = ce6_raw['DateTime'].max() - ce6_raw['DateTime'].min()
display(ce6_hrs_range)

### 4.	Finally, combine the time series information from the two datasets together to see how well correlated heavy job-completion load is with the unresponsiveness, and to see if there is some threshold of job completions per hour that generally results in unresponsiveness.